In [ ]:
!pip install "gensim>=4.0.0"

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

In [ ]:
import regex as re
from spacy.lang.de.stop_words import STOP_WORDS as stop_words
gensim_words = [[w for w in re.split(r'[\\|\\#]', doc.lower()) if w not in stop_words]
                    for doc in df["nav"]]

In [ ]:
from gensim.models import Word2Vec
w2v = Word2Vec(gensim_words, min_count=5)

In [ ]:
w2v.wv.save_word2vec_format("heise-articles-2020.w2v")

In [ ]:
w2v.wv.most_similar(positive=["java"], topn=10)

In [ ]:
w2v.wv.most_similar(positive=["microsoft"], topn=10)

In [ ]:
w2v.wv.most_similar(positive=["corona"], topn=10)

In [ ]:
pd.DataFrame(w2v.wv.most_similar(positive=["apple", "android"], 
                                   negative=["google"],  topn=10))

In [ ]:
pd.DataFrame(w2v.wv.most_similar(positive=["microsoft", "android"], 
                                   negative=["google"],  topn=10))

In [ ]:
from gensim.models import Phrases
entity_transformer = Phrases(gensim_words)
w2vp = Word2Vec(entity_transformer[gensim_words], min_count=5)

In [ ]:
w2vp.wv.most_similar(positive=["corona"], topn=10)

In [ ]:
w2vp.wv.most_similar(positive=["java"], topn=10)